In [1]:
import os
from glob import glob
from datetime import datetime

from scipy import interpolate
from pytmatrix import orientation, radar, tmatrix_aux, refractive
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix.tmatrix import TMatrix, Scatterer
from pytmatrix.tmatrix_psd import TMatrixPSD, GammaPSD

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
from tqdm import tqdm

import common

In [2]:
file_list = sorted(glob('/g/data/kl02/jss548/PST/disdrometer/Broadmeadows/*.csv'))


In [3]:
def broadmeadows_disdro_to_radar_moments(file_list):
    
    #init
    empty_value = -9.999
    mean_diam_drop_class = np.array([0.062, 0.187, 0.312, 0.437, 0.562, 0.687, 0.812, 
                                      0.937, 1.062, 1.187, 1.375, 1.625, 1.875, 2.125,
                                      2.375, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 6.5, 
                                      7.5, 8.5, 9.5, 11, 13, 15, 17, 19, 21.5, 24.5])
    
    #init lists
    DBZ_list = []
    ZDR_list = []
    KDP_list = []
    ATTEN_list = []
    RAIN_list = []
    TIME_list = []
    
    #parse all text files
    data_line_len = 50
    dsd_dictlist = []
    print('processing', infile)
    #init lists
    dt_list = []
    nd_list = []
    rr_list = []
    #find total number of times
    num_lines = sum(1 for line in open(infile,'r'))
    #read file
    with open(infile) as f:
        #for every line
        for count, line in tqdm(enumerate(f), total = num_lines):
            if count>0: # skip first line
                #split line
                data_read = line.split(',')
                #only process data string lines
                if len(data_read) >= 1000:
                    #append to lists
                    dt_list.append(datetime.strptime(data_read[0],'%Y-%m-%d %H:%M:%S'))
                    nd_list.append(list(map(float,data_read[11:43]))) #Field N(d) 90
                    rr_list.append(float(data_read[1]))
                else:
                    #catch short lists
                    print('error', data_read)
                    break
                        
    for i, nd in tqdm(enumerate(nd_list), total=len(nd_list)):
        #convert list to array
        nd_array = np.array(nd)
        #replace empty value with zeros
        nd_array[nd_array==empty_value] = 0
        #skip when no data is present
        if np.sum(nd_array) == 0:
            continue

        #calc radar moments
        normal_nd_array = nd_array/np.gradient(mean_diam_drop_class)
        print(np.shape(mean_diam_drop_class))
        print(np.shape(normal_nd_array))
        dbz, zdr, kdp, atten_spec = common.scatter_off_2dvd_packed(mean_diam_drop_class, normal_nd_array, scatterer)

        #append outputs
        DBZ_list.append(dbz)
        ZDR_list.append(zdr)
        KDP_list.append(kdp)
        ATTEN_list.append(atten_spec)
        RAIN_list.append(rr_list[i])
        TIME_list.append(dt_list[i])

    return DBZ_list, ZDR_list, KDP_list, ATTEN_list, TIME_list, RAIN_list

In [4]:
band = 'C'
cant = 10 #wdith of canting angle distribution
temperature = 0 #used for refractive calculations

# Main T-matrix parameters initialisation
# Radar band in mm.
if band == 'S':
    radar_band = tmatrix_aux.wl_S
elif band == 'C':
    radar_band = tmatrix_aux.wl_C
# Scatterer class from pytmatrix
# tmatrix_aux.wl_C is the wavelength in mm
# refractive.m_w_10C is a dictionnary containing the refractive index (at 10C) for different wavelength.
if temperature == 0:
    scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_0C[radar_band])
elif temperature == 10:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_10C[radar_band])
elif temperature == 20:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_20C[radar_band])
scatterer.or_pdf = orientation.gaussian_pdf(cant)
scatterer.orient = orientation.orient_averaged_fixed    

# PSDIntegrator classfrom pytmatrix
scatterer.psd_integrator = PSDIntegrator()    

# Defining the axis ratio of drops.
scatterer.psd_integrator.D_max = 8
scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)

##### !!!!HERE!!!! ######
scatterer.psd_integrator.axis_ratio_func = common.bzv_model
##### !!!!HERE!!!! ######

scatterer.psd_integrator.init_scatter_table(scatterer)

In [5]:
#init lists
DBZ_total = []
ZDR_total = []
KDP_total = []
ATTEN_total = []
TIME_total = []
RAIN_total = []

for infile in file_list:
    #load file and run pytmatrix
    DBZ_out, ZDR_out, KDP_out, ATTEN_out, TIME_out, RAIN_out = broadmeadows_disdro_to_radar_moments(infile)
    DBZ_total.extend(DBZ_out)
    ZDR_total.extend(ZDR_out)
    KDP_total.extend(KDP_out)
    ATTEN_total.extend(ATTEN_out)
    TIME_total.extend(TIME_out)
    RAIN_total.extend(RAIN_out)
    
dbz_array = np.array(DBZ_total)
zdr_array = np.array(ZDR_total)
kdp_array = np.array(KDP_total)
att_array = np.array(ATTEN_total)
time_array = np.array(TIME_total)
rain_array = np.array(RAIN_total)

zdr_db_array = 10*np.log10(zdr_array)

processing /g/data/kl02/jss548/PST/disdrometer/Broadmeadows/OTT1_2015.csv


  0%|          | 0/514838 [00:00<?, ?it/s]/home/548/jss548/dev/projects/PST/DSD/common.py:84: RuntimeWarning: invalid value encountered in log10
  dbz = 10*np.log10(radar.refl(scatterer))  # in dBZ
  1%|          | 3346/514838 [00:00<00:15, 33038.77it/s]

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,

  2%|▏         | 9081/514838 [00:00<00:48, 10367.90it/s]

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,

...

 19%|█▉        | 100248/514838 [00:05<00:19, 21245.48it/s]

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,

 20%|██        | 104938/514838 [00:06<00:24, 16915.63it/s]

(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,)
(32,

KeyboardInterrupt: 

In [14]:
#convert zdr to log space
dbz_mask1 = np.logical_or(np.isinf(dbz_array), np.isnan(dbz_array))
dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)
dbz_mask = np.logical_or(dbz_mask1, dbz_mask2)

#apply filters
dbz_array_filt = dbz_array[~dbz_mask]
rain_array_filt = rain_array[~dbz_mask]
zdr_array_filt = zdr_array[~dbz_mask]
kdp_array_filt = kdp_array[~dbz_mask]
att_array_filt = att_array[~dbz_mask]
time_array_filt = time_array[~dbz_mask]

#convert zdr into log space
zdr_db_array_filt = 10*np.log10(zdr_array_filt)

<ipython-input-14-30252da145b1>:3: RuntimeWarning: invalid value encountered in less
  dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)
<ipython-input-14-30252da145b1>:3: RuntimeWarning: invalid value encountered in greater
  dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)


In [15]:
np.savez(f'pytmatrix_out/broadmeadows_radarsim_{band}_{cant}deg_{temperature}C.npz', dbz_array=dbz_array_filt, rain_array=rain_array_filt, zdr_db_array=zdr_db_array_filt, kdp_array=kdp_array_filt, att_array=att_array_filt, time_array = time_array_filt)